In [4]:
import gluonbook as gb
from mxnet import gluon ,init,nd
from mxnet.gluon import nn
import mxnet as mx

## Dataset

In [5]:
batch_size=128
train_iter,test_iter=gb.load_data_fashion_mnist(batch_size,resize=96)

## model(googlenet)

In [89]:
class Inception(nn.Block):
    def __init__(self,c1,c2,c3,c4):
        super(Inception,self).__init__()
        
        self.p1_1=nn.Conv2D(c1,kernel_size=1,activation='relu')
        
        self.p2_1=nn.Conv2D(c2[0],kernel_size=1,activation='relu')
        self.p2_2=nn.Conv2D(c2[1],kernel_size=3,padding=1,activation='relu')
        
        self.p3_1=nn.Conv2D(c3[0],kernel_size=1,activation='relu')
        self.p3_2=nn.Conv2D(c3[1],kernel_size=5,padding=2,activation='relu')
        
        self.p4_1=nn.MaxPool2D(pool_size=3,padding=1,strides=1)
        self.p4_2=nn.Conv2D(c4,kernel_size=1,activation='relu')
    def forward(self,X):
        p1=self.p1_1(X)
        p2=self.p2_2(self.p2_1(X))
        p3=self.p3_2(self.p3_1(X))
        p4=self.p4_2(self.p4_1(X))
        return nd.concat(p1,p2,p3,p4,dim=1)

In [90]:
b1=nn.Sequential()
b1.add(nn.Conv2D(64,kernel_size=7,strides=2,padding=3,activation='relu'),
      nn.MaxPool2D(pool_size=3,padding=1,strides=2))

In [91]:
b2=nn.Sequential()
b2.add(nn.Conv2D(64,kernel_size=1),
      nn.Conv2D(192,kernel_size=3,padding=1),
      nn.MaxPool2D(pool_size=3,strides=2,padding=1))

In [92]:
b3=nn.Sequential()
b3.add(Inception(64,(96,128),(16,32),32),
      Inception(128,(128,192),(32,96),64),
      nn.MaxPool2D(pool_size=3,strides=2,padding=1))

In [93]:
b4=nn.Sequential()
b4.add(Inception(192,(96,208),(16,48),64),
      Inception(160,(112,224),(24,64),64),
      Inception(128,(128,256),(24,64),64),
      Inception(112,(144,288),(32,64),64),
      Inception(256,(160,320),(32,128),128),
      nn.MaxPool2D(pool_size=3,padding=1,strides=2))

In [94]:
b5=nn.Sequential()
b5.add(Inception(256,(160,320),(32,128),128),
      Inception(384,(192,384),(48,128),128),
      nn.GlobalAvgPool2D())

In [95]:
net=nn.Sequential()
net.add(b1,b2,b3,b4,b5,nn.Dense(10))

## train & eva

In [101]:
lr,num_epochs,ctx=0.1,5,mx.gpu()
net.initialize(force_reinit=True,ctx=ctx,init=init.Xavier())
trainer=gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':lr})
gb.train_ch5(net,train_iter,test_iter,batch_size,trainer,ctx,num_epochs)

training on gpu(0)
epoch 1, loss 2.0958, train acc 0.228, test acc 0.670, time 244.4 sec
epoch 2, loss 0.7170, train acc 0.728, test acc 0.832, time 229.5 sec
epoch 3, loss 0.4702, train acc 0.825, test acc 0.862, time 230.6 sec
epoch 4, loss 0.3786, train acc 0.855, test acc 0.876, time 230.9 sec
epoch 5, loss 0.3390, train acc 0.872, test acc 0.885, time 230.9 sec
